In [ ]:
#Required Library
!pip install torch torchvision torchaudio einops timm pillow
!pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/accelerate
!pip install git+https://github.com/huggingface/diffusers
!pip install hugging face_hub
!pip install sentencepiece bitsandbytes protobuf decord

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-0qdlxmla
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-0qdlxmla
  Resolved https://github.com/huggingface/transformers to commit 4c1388f48e72dde96b666f8fe24f74bf4056e410
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10100594 sha256=bb08e02f258c254794d63ef59f34886f0cb992da612a330d7f81bed475a9cf5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-42z8jcm8/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
  Cloning https://github.com/huggingfac

In [ ]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 39.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.0.post2-cp310-cp310-linux_x86_64.whl size=183291101 sha256=16a849d51b95cf8e47a6e6cd36826e9ffbbc068a8546e7e3501a598bd70905a6
  Stored in directory: /root/.cache/pip/wheels/bf/e3/ed/5e845387d52f2debd1bafb847bf3d774d3f0a3c8e31b1dc948
Successfully built flash-attn


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

True
Tesla T4
12.1


In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize model with "eager" attention
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    torch_dtype=torch.bfloat16,  # Adjust precision for GPU efficiency
    _attn_implementation="eager",  # Use eager attention to avoid Flash Attention issues
).to(DEVICE)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/429 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

...

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


config.json:   0%|          | 0.00/7.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
# Load images
image1 = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
image2 = load_image("https://huggingface.co/spaces/merve/chameleon-7b/resolve/main/bee.jpg")

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "Can you describe the two images?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = inputs.to(DEVICE)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])


User:<image>Can you describe the two images?
Assistant: The first image shows a statue of the Statue of Liberty in front of a cityscape with a body of water and a clear sky. The second image shows a bee on a pink flower surrounded by other flowers and plants.


In [ ]:
# Load images # Replace image path
image1 = load_image("/content/1_GZ2oHiedOWDoqTZV6ady3g-ezgif.com-webp-to-jpg-converter.jpg")
image2 = load_image("/content/1_TRwyjqqAlwcElS4emWRXrg-ezgif.com-webp-to-jpg-converter.jpg")

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image1"},
            {"type": "text", "text": "Can you describe the the image?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to(DEVICE)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])


ValueError: The total number of <image> tokens in the prompts should be the same as the number of images passed. Found 0 <image> tokens and 1 images.

In [ ]:
def generate_image_description(image, prompt, model, processor, device):
    """
    This function generates a description for the passed image based on the given prompt.

    Parameters:
    - image: The input image to be described.
    - prompt: The text prompt to guide the image description generation.
    - model: The model used for generation.
    - processor: The processor used for tokenization and preparing inputs.
    - device: The device on which the model should run (e.g., 'cuda' or 'cpu').

    Returns:
    - The generated description for the image.
    """
    # Insert the <image> token in the prompt
    prompt_with_image = f"{prompt} <image>"

    # Create input messages with the prompt and image
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_with_image}
            ]
        },
    ]

    # Prepare the input based on the message format
    prompt_with_image = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt_with_image, images=[image], return_tensors="pt")
    inputs = inputs.to(device)

    # Generate outputs from the model
    generated_ids = model.generate(**inputs, max_new_tokens=500)

    # Decode the generated ids into text
    generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)

    return generated_texts[0]

In [ ]:

# Example usage:
image1 = load_image("/content/1_GZ2oHiedOWDoqTZV6ady3g-ezgif.com-webp-to-jpg-converter.jpg")
image2 = load_image("/content/1_TRwyjqqAlwcElS4emWRXrg-ezgif.com-webp-to-jpg-converter.jpg")

# Assuming image1 is your image, and you have the model, processor, and device setup
image_description = generate_image_description(image2, "Are there eggs available on the  shopping cart?", model, processor, DEVICE)
print(image_description)


User: Are there eggs available on the  shopping cart? <image>Assistant: I'm ripe peaches are in the cart.


In [ ]:
# Example usage:
image3 = load_image("/content/basic-invoice-template.png")
image4 = load_image("/content/A-sample-prescription-image-in-grayscale-version.png")

# Assuming image1 is your image, and you have the model, processor, and device setup
image_description = generate_image_description(image3, "Extract text from prescription image?", model, processor, DEVICE)
print(image_description)


User: Extract text from prescription image? <image>Assistant: [Company Name]
[Street Address]
[City, ST, ZIP]
Phone: (000) 000-0000

INVOICE
[Invoice #]
[Date]

BILL TO
[Name]
[Company Name]
[Street Address]
[City, ST, ZIP]
[Email Address]

DESCRIPTION
DESCRIPTION
AMOUNT
Service Fee
200.00
Labor: 5 hours at $75/hr
375.00
New client discount
(50.00)
Tax (4.25% after discount)
26.56

TOTAL
$ 551.56

Thank you for your business!
